## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-03-54-00 +0000,wsj,Trump’s Big Tariff Task in Asia Is to Close th...,https://www.wsj.com/economy/trade/trump-tariff...
1,2025-10-28-03-52-23 +0000,nypost,More than 20 ‘ill-prepared’ hikers — some firs...,https://nypost.com/2025/10/27/us-news/more-tha...
2,2025-10-28-03-44-55 +0000,nyt,Live Updates: Hurricane Melissa Turns Toward J...,https://www.nytimes.com/live/2025/10/21/weathe...
3,2025-10-28-03-15-28 +0000,startribune,Minnesota United takes Game 1 of MLS playoff s...,https://www.startribune.com/minnesota-united-t...
4,2025-10-28-03-10-40 +0000,startribune,Mayoral candidates keep the gloves on during s...,https://www.startribune.com/minneapolis-mayor-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,50
86,new,18
71,china,16
455,he,13
239,milei,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
272,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,141
58,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,99
8,2025-10-28-02-47-25 +0000,wapo,Trump signs rare earth minerals deal with Japa...,https://www.washingtonpost.com/politics/2025/1...,98
244,2025-10-27-12-47-34 +0000,cbc,When will Trump's tariff increase hit Canada? ...,https://www.cbc.ca/news/world/trump-tariff-can...,95
253,2025-10-27-11-47-07 +0000,nypost,"Trump warns New Jersey faces hotbed of crime, ...",https://nypost.com/2025/10/27/us-news/trump-wa...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
272,141,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
280,75,2025-10-27-09-01-23 +0000,wsj,"Stock Market News, Oct. 27, 2025: Dow Hits Ne...",https://www.wsj.com/livecoverage/stock-market-...
92,56,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
55,55,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
187,46,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...
202,41,2025-10-27-15-42-00 +0000,nypost,Treasury Secretary Bessent says he felt ‘pain’...,https://nypost.com/2025/10/27/business/treasur...
300,39,2025-10-27-06-46-40 +0000,nypost,Trump congratulates Argentina’s Javier Milei a...,https://nypost.com/2025/10/27/world-news/argen...
160,36,2025-10-27-18-01-17 +0000,nypost,Andrew Cuomo narrows gap against Zohran Mamdan...,https://nypost.com/2025/10/27/us-news/andrew-c...
189,35,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
270,31,2025-10-27-09-52-22 +0000,bbc,UN pleads for safe passage for civilians trapp...,https://www.bbc.com/news/articles/c4gw8w73lngo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
